In [ ]:
import sys
print(sys.path)

In [ ]:
import os

print (os.environ)

In [ ]:
import pyspark
import random

sc = pyspark.SparkContext(appName="Pi")

num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()

pi = 4 * count / num_samples
print(pi)

sc.stop()

In [ ]:
sc.stop()

In [ ]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sc= SparkContext()
sqlContext = SQLContext(sc)
house_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('boston.csv')
house_df.take(1)

In [ ]:
import six
for i in house_df.columns:
    if not( isinstance(house_df.select(i).take(1)[0][0], six.string_types)):
        print( "Correlation to MV for ", i, house_df.stat.corr('MV',i))

In [ ]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PT', 'B', 'LSTAT'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'MV'])
vhouse_df.show(3)

In [ ]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]

In [ ]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='MV', maxIter=10, regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

## Cox model

In [ ]:
import s3fs
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
# csv file
df_all = pd.read_csv("s3://eqrs-ngmc-datascience/Datascience/x.csv")
df = df_all#[:50000]

# as.Date(data$strr_pyf_start, format="%m/%d/%Y")
df['strr_pyf_start'] = pd.to_datetime(df['strr_pyf_start']).dt.strftime('%m/%d/%Y')
# df['provfs'] = df['provfs'].astype(str)
df['provfs'] = pd.to_numeric(df['provfs'], downcast='integer', errors='coerce').fillna(0)
df['provfs'] = df['provfs'].astype(int)
col_names = df.columns

# sorting with three columns
z_pd = df.to_records()
z_pd.sort(order=["provfs", "ptnt_id", "year"])
d2 = pd.DataFrame(z_pd)

classnames = (
"pdiab", "pdismiss", "pnhprev", "bmi_msg", "ashd1", "othcardiac1",
"carfail", "noambul", "pulmon", "notrans", "cancer", "diabetes",
"pvasc", "cva", "smoke", "alcoh", "drug", "inci_one", "inci_miss"
)

#R     z <- match(classnames, names(data2))
z = [ list(d2.columns).index(x) if x in d2.columns else None for x in  classnames]

for i in range(0, len(z)):
    col_mean = float(d2.iloc[:, [z[i]]].mean())
    if(col_mean<0.001 or col_mean>0.999):
        print("WARNING: Covariate", classnames[i],"has less than 0.1% variation.  The distribution of this covariate is", col_mean)
    else:
        print("WARNING: Covariate", classnames[i], "has missing values.")



cols = ["ptnt_id", "provfs", "t_trans", "strr_pyf_start", "wt_trans", "ot_trans","pdiab", 
        "pdismiss", "pnhprev",  "logbmi", "bmi_msg", "agecat6", "year", "pyf_period_esrd",  
        "t_start", "t_stop", "ashd1", "othcardiac1", "carfail", "noambul",  "pulmon", 
        "notrans", "cancer", "diabetes", "pvasc", "cva",  "smoke", "alcoh", "drug", "inci_one", "inci_miss"]
z2 = d2[cols]

missing_z2 = z2.isna()

if sum(missing_z2.sum(axis=0))>0:
  print("Warning: There are", sum(missing_z2.sum(axis=0)), "rows with missing data")
  print("Any rows with missing data will be deleted")

allnames= ["ptnt_id","provfs","t_trans", "strr_pyf_start","wt_trans","ot_trans","pdiab","pdismiss",
    "pnhprev","logbmi","bmi_msg","agecat6","year","pyf_period_esrd","t_start","t_stop","ashd1", 
    "othcardiac1", "carfail","noambul", "pulmon", "notrans", "cancer", "diabetes",
    "pvasc", "cva", "smoke","alcoh", "drug","inci_one","inci_miss",
    "strr_period", "pstrr", "trans_yar", "trans_dar"]


data_sub = d2[allnames]
data_sub_complete=data_sub

########################################################################################
#  SECTION 7 (ISSUE #3): CREATE TRANSFUSION EVENT FLAG                                 #
#    -CHECK FOR SMALL NUMBER OF EVENTS                                                 #                                           
########################################################################################
data_sub_complete['t_trans0'] = np.where(data_sub_complete['t_trans']>0, 1, 0)
data_sub_complete['t_trans']=data_sub_complete['t_trans'].apply(lambda x: 1 if x > 0 else 0).copy()
# data_sub_complete.loc[data_sub_complete.t_trans > 1, 't_trans'] = 1
# data_sub_complete.loc[data_sub_complete.t_trans <= 0, 't_trans'] = 0

########################################################################################
#   SECTION 8: SUBSET DATA TO THOSE WITH AT LEAST 1 DAY AT RISK                        #
#     -ALSO SUBSET TO THOSE WITH APPROPRIATE AGE AND ESRD CATEGORIES                   #
#   NOTE: THIS IS MOSTLY DONE JUST TO BE PRECAUTIOUS FOR TEST DATA                     #
########################################################################################
data_sub_complete2 = data_sub_complete[(data_sub_complete['pyf_period_esrd']>0) & (data_sub_complete['agecat6']!=1) & (data_sub_complete['trans_dar']>0)]

########################################################################################
#   SECTION 9 (ISSUE #5): CHECK FOR LINEARLY DEPENDENT COVARIATES                      #
#      NOTE: THIS WON'T PREVENT MODEL FROM RUNNING                                     #
########################################################################################

z_pd = data_sub_complete2.to_records()
z_pd.sort(order=["provfs", "ptnt_id", "year", "strr_pyf_start"])
data_sub_sort = pd.DataFrame(z_pd)
data_chk_rank = data_sub_sort.drop(["strr_pyf_start","provfs","ptnt_id"], axis = 1) 

########################################################################################
#   SECTION 10: RUN DATA THROUGH TWO COX PROPORTIONAL HAZARDS MODELS                   #
########################################################################################
data_sub_sort['year'] = pd.Categorical(data_sub_sort['year'], ordered=False)
data_sub_sort['pdismiss'] = pd.Categorical(data_sub_sort['pdismiss'], categories=[0, 1], ordered=False)
data_sub_sort['pnhprev'] = pd.Categorical(data_sub_sort['pnhprev'], categories=[0, 1], ordered=False)
data_sub_sort['bmi_msg'] = pd.Categorical(data_sub_sort['bmi_msg'], categories=[0, 1], ordered=False)
data_sub_sort['ashd1'] = pd.Categorical(data_sub_sort['ashd1'], categories=[0, 1], ordered=False)
data_sub_sort['othcardiac1'] = pd.Categorical(data_sub_sort['othcardiac1'], categories=[0, 1], ordered=False)
data_sub_sort['carfail'] = pd.Categorical(data_sub_sort['carfail'], categories=[0, 1], ordered=False)
data_sub_sort['noambul'] = pd.Categorical(data_sub_sort['noambul'], categories=[0, 1], ordered=False)
data_sub_sort['pulmon'] = pd.Categorical(data_sub_sort['pulmon'], categories=[0, 1], ordered=False)
data_sub_sort['notrans'] = pd.Categorical(data_sub_sort['notrans'], categories=[0, 1], ordered=False)
data_sub_sort['cancer'] = pd.Categorical(data_sub_sort['cancer'], categories=[0, 1], ordered=False)
data_sub_sort['diabetes'] = pd.Categorical(data_sub_sort['diabetes'], categories=[0, 1], ordered=False)
data_sub_sort['pvasc'] = pd.Categorical(data_sub_sort['pvasc'], categories=[0, 1], ordered=False)
data_sub_sort['cva'] = pd.Categorical(data_sub_sort['cva'], categories=[0, 1], ordered=False)
data_sub_sort['smoke'] = pd.Categorical(data_sub_sort['smoke'], categories=[0, 1], ordered=False)
data_sub_sort['alcoh'] = pd.Categorical(data_sub_sort['alcoh'], categories=[0, 1], ordered=False)
data_sub_sort['drug'] = pd.Categorical(data_sub_sort['drug'], categories=[0, 1], ordered=False)
data_sub_sort['inci_one'] = pd.Categorical(data_sub_sort['inci_one'], categories=[0, 1], ordered=False)
data_sub_sort['inci_miss'] = pd.Categorical(data_sub_sort['inci_miss'], categories=[0, 1], ordered=False)
data_sub_sort['smoke'] = pd.Categorical(data_sub_sort['smoke'], categories=[0, 1], ordered=False)

data_sub_sort['provfs'] = pd.Categorical(data_sub_sort['provfs'], ordered=False)

# coxph_control <- coxph.control(eps = 1e-8)
# data_sub_sort.transform(lambda x: x + 1)

data_model = data_sub_sort

#######################################################################################################################################################
# Run Stage-1 Cox Model
######################################################################################################################################################
import h2o
from h2o.estimators.coxph import H2OCoxProportionalHazardsEstimator
h2o.init()

data_modelt_Full_Sample=data_model
data_modelt_Full_Sample.reset_index(drop=True, inplace=True)

data_modelt_Full_Sample_hex = h2o.H2OFrame(data_modelt_Full_Sample)
data_modelt_Full_Sample_hex[data_modelt_Full_Sample_hex["provfs"].isna(), "provfs"] = 0
# data_modelt_Full_Sample_hex["provfs"] = data_modelt_Full_Sample_hex["provfs"].ascharacter()
# data_modelt_Full_Sample_hex["provfs"] = data_modelt_Full_Sample_hex["provfs"].asfactor()
predictorsSt = list(["agecat6", "pdiab", "pdismiss","notrans","cancer",
                "diabetes","pvasc","year", "pnhprev", "logbmi", 
                "bmi_msg","cva","smoke","alcoh","drug",
                "inci_one", "ashd1", "pulmon","inci_miss","year",
                "othcardiac1", "carfail", "noambul", "pulmon"])

interaction_pairs = [   ("agecat6","pdiab"), 
                        ("pdiab", "pyf_period_esrd")]

strr_h2o_moodel = H2OCoxProportionalHazardsEstimator(
    start_column="t_start",
    stop_column="t_stop",
    offset_column="ot_trans",
    ties="breslow",
#     stratify_by=["provfs"],
#     interaction_pairs=interaction_pairs,
    )

strr_h2o_moodel.train(x=predictorsSt,
                y="t_trans0",
                training_frame=data_modelt_Full_Sample_hex)



###########################
# writing to csv
# pd.DataFrame(data_modelt_Full_Sample_hex["provfs"]).to_csv('provfs.csv')

data_modelt_Full_Sample_hex['provfs'].types

strr_h2o_moodel.coefficients_table
data_modelt_Full_Sample_hex['t_trans0'].types
data_modelt_Full_Sample_hex["t_trans0"]
len(data_modelt_Full_Sample_hex.col_names)
data_modelt_Full_Sample_hex.types

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,2 hours 23 mins
H2O_cluster_timezone:,UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.3
H2O_cluster_version_age:,2 months and 4 days
H2O_cluster_name:,H2O_from_python_root_z8cwc4
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,25.25 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,32
H2O_cluster_status:,"locked, healthy"


### S3 CSV file read

In [ ]:
import s3fs
import pandas as pd
import numpy as np
pd.options.display.max_columns = None
# csv file
df_all = pd.read_csv("s3://eqrs-ngmc-datascience/Datascience/x.csv")

# parquet file
# df = pd.read_parquet('s3://{bucket_name}/{path_to_file}')

In [ ]:
df = df_all#[:50000]
df.head()

In [ ]:
# as.Date(data$strr_pyf_start, format="%m/%d/%Y")
df['strr_pyf_start'] = pd.to_datetime(df['strr_pyf_start']).dt.strftime('%m/%d/%Y')
# df['provfs'] = df['provfs'].astype(str)
df['provfs'] = pd.to_numeric(df['provfs'], downcast='integer', errors='coerce').fillna(0)
df['provfs'] = df['provfs'].astype(int)

In [ ]:
col_names = df.columns
df["provfs"].loc[0]

In [ ]:
# sorting with three columns
z_pd = df.to_records()
z_pd.sort(order=["provfs", "ptnt_id", "year"])
d2 = pd.DataFrame(z_pd)
d2

In [ ]:
# A = np.array(df["provfs"].astype(int)).argsort()
# A

In [ ]:
  classnames = (
    "pdiab", "pdismiss", "pnhprev", "bmi_msg", "ashd1", "othcardiac1",
    "carfail", "noambul", "pulmon", "notrans", "cancer", "diabetes",
    "pvasc", "cva", "smoke", "alcoh", "drug", "inci_one", "inci_miss"
  )

In [ ]:
#R     z <- match(classnames, names(data2))
z = [ list(d2.columns).index(x) if x in d2.columns else None for x in  classnames]

In [ ]:
#R
#  for (i in 1:length(z)){
#     if(is.na(mean(data2[,z[i]]))==FALSE){
#       if(mean(data2[,z[i]])<0.001 || mean(data2[,z[i]])>0.999 ){
#         print(paste("WARNING: Covariate",toupper(classnames[i]),"has less than 0.1% variation.  The distribution of this covariate is", mean(data2[,z[i]])))
#       }
#     }else
#     {
#       print(paste("WARNING: Covariate", toupper(classnames[i]), "has missing values."))
#     }
#   }

for i in range(0, len(z)):
    col_mean = float(d2.iloc[:, [z[i]]].mean())
    if(col_mean<0.001 or col_mean>0.999):
        print("WARNING: Covariate", classnames[i],"has less than 0.1% variation.  The distribution of this covariate is", col_mean)
    else:
        print("WARNING: Covariate", classnames[i], "has missing values.")

    

In [ ]:
###   adjustment variables
# R  z2<-cbind(data2$ptnt_id, data2$provfs, data2$t_trans, data2$strr_pyf_start,
#           data2$wt_trans, data2$ot_trans,data2$pdiab, data2$pdismiss, data2$pnhprev, 
#           data2$logbmi, data2$bmi_msg, data2$agecat6, data2$year, data2$pyf_period_esrd, 
#           data2$t_start, data2$t_stop, data2$ashd1, data2$othcardiac1, data2$carfail, data2$noambul, 
#           data2$pulmon, data2$notrans, data2$cancer, data2$diabetes, data2$pvasc, data2$cva, 
#           data2$smoke, data2$alcoh, data2$drug, data2$inci_one, data2$inci_miss)
cols = ["ptnt_id", "provfs", "t_trans", "strr_pyf_start", "wt_trans", "ot_trans","pdiab", 
        "pdismiss", "pnhprev",  "logbmi", "bmi_msg", "agecat6", "year", "pyf_period_esrd",  
        "t_start", "t_stop", "ashd1", "othcardiac1", "carfail", "noambul",  "pulmon", 
        "notrans", "cancer", "diabetes", "pvasc", "cva",  "smoke", "alcoh", "drug", "inci_one", "inci_miss"]
z2 = d2[cols]
z2

In [ ]:
missing_z2 = z2.isna()

In [ ]:
# R
# if(sum(rowSums(missing_z2)>0)){
#   print(paste("Warning: There are", sum(rowSums(missing_z2)>0), "rows with missing data"))
#   print(paste("Any rows with missing data will be deleted"))
# }
if sum(missing_z2.sum(axis=0))>0:
  print("Warning: There are", sum(missing_z2.sum(axis=0)), "rows with missing data")
  print("Any rows with missing data will be deleted")


In [ ]:
# allnames=
#   c("ptnt_id","provfs","t_trans", "strr_pyf_start","wt_trans","ot_trans","pdiab","pdismiss",
#     "pnhprev","logbmi","bmi_msg","agecat6","year","pyf_period_esrd","t_start","t_stop","ashd1", 
#     "othcardiac1", "carfail","noambul", "pulmon", "notrans", "cancer", "diabetes",
#     "pvasc", "cva", "smoke","alcoh", "drug","inci_one","inci_miss",
#     "strr_period", "pstrr", "trans_yar", "trans_dar")

# data_sub <- data2[ ,allnames]
# data_sub_complete<-data_sub                          #CHANGED CODE LINE HERE

allnames= ["ptnt_id","provfs","t_trans", "strr_pyf_start","wt_trans","ot_trans","pdiab","pdismiss",
    "pnhprev","logbmi","bmi_msg","agecat6","year","pyf_period_esrd","t_start","t_stop","ashd1", 
    "othcardiac1", "carfail","noambul", "pulmon", "notrans", "cancer", "diabetes",
    "pvasc", "cva", "smoke","alcoh", "drug","inci_one","inci_miss",
    "strr_period", "pstrr", "trans_yar", "trans_dar"]


data_sub = d2[allnames]
data_sub_complete=data_sub
data_sub_complete

In [ ]:
########################################################################################
#  SECTION 7 (ISSUE #3): CREATE TRANSFUSION EVENT FLAG                                 #
#    -CHECK FOR SMALL NUMBER OF EVENTS                                                 #                                           
########################################################################################
#create flag for transfusions==>will be used in model as event
# data_sub_complete$t_trans0 <- ifelse(data_sub_complete$t_trans>0, 1, 0)

data_sub_complete['t_trans0'] = np.where(data_sub_complete['t_trans']>0, 1, 0)
data_sub_complete['t_trans']=data_sub_complete['t_trans'].apply(lambda x: 1 if x > 0 else 0).copy()
# data_sub_complete.loc[data_sub_complete.t_trans > 1, 't_trans'] = 1
# data_sub_complete.loc[data_sub_complete.t_trans <= 0, 't_trans'] = 0
data_sub_complete

In [ ]:
sum(data_sub_complete['provfs']==0)

In [ ]:
########################################################################################
#   SECTION 8: SUBSET DATA TO THOSE WITH AT LEAST 1 DAY AT RISK                        #
#     -ALSO SUBSET TO THOSE WITH APPROPRIATE AGE AND ESRD CATEGORIES                   #
#   NOTE: THIS IS MOSTLY DONE JUST TO BE PRECAUTIOUS FOR TEST DATA                     #
########################################################################################
#subset to values that should not be in data, and to those with at least 1 day at risk

#   data_sub_complete2 <- data_sub_complete[ which(data_sub_complete$pyf_period_esrd>0 &
#                                                    data_sub_complete$agecat6 != 1 & data_sub_complete$trans_dar>0), ]
  
#   if(TO_LOG==1){
#     cat("Step 1 of 4 completed - data cleaned up", file=log_prog_strr, sep="\n")
#     cat("Step 1 of 4 completed - data cleaned up")
#   }
#   if(TO_LOG!=1){
#     cat("Step 1 of 4 completed - data cleaned up")
#   }

# data_sub_sort=data_sub_complete2[order(data_sub_complete2$provfs,data_sub_complete2$ptnt_id,
#                                        factor(data_sub_complete2$year),data_sub_complete2$strr_pyf_start),] 

data_sub_complete2 = data_sub_complete[(data_sub_complete['pyf_period_esrd']>0) & (data_sub_complete['agecat6']!=1) & (data_sub_complete['trans_dar']>0)]
data_sub_complete2

In [ ]:
########################################################################################
#   SECTION 9 (ISSUE #5): CHECK FOR LINEARLY DEPENDENT COVARIATES                      #
#      NOTE: THIS WON'T PREVENT MODEL FROM RUNNING                                     #
########################################################################################
# data_chk_rank <- data_sub_sort[ ,-which(names(data_sub_sort) %in% c("strr_pyf_start","provfs","ptnt_id"))]

# if(qr(data_chk_rank)$rank!=ncol(data_chk_rank)){
#   print(paste("There is rank deficiency in covariates."))
#   print(paste("Any collinear covariates are skipped over."))
#   print(paste("This results in value of 'NA' for estimated coefficient. "))
# }
z_pd = data_sub_complete2.to_records()
z_pd.sort(order=["provfs", "ptnt_id", "year", "strr_pyf_start"])
data_sub_sort = pd.DataFrame(z_pd)
data_chk_rank = data_sub_sort.drop(["strr_pyf_start","provfs","ptnt_id"], axis = 1) 
data_chk_rank

In [ ]:
########################################################################################
#   SECTION 10: RUN DATA THROUGH TWO COX PROPORTIONAL HAZARDS MODELS                   #
########################################################################################

# #TRANSFORM CLASS VARIABLES INTO FACTORS, SO THAT R TREATS THESE APPROPRIATELY
# data_model <- transform(data_sub_sort, 
#                         year=factor(year),
#                         pdismiss=factor(pdismiss, levels=c(0,1)),
#                         pnhprev=factor(pnhprev, levels=c(0,1)), bmi_msg=factor(bmi_msg, levels=c(0,1)),
#                         ashd1=factor(ashd1, levels=c(0,1)), othcardiac1=factor(othcardiac1, levels=c(0,1)),
#                         carfail=factor(carfail, levels=c(0,1)), noambul=factor(noambul, levels=c(0,1)),
#                         pulmon=factor(pulmon, levels=c(0,1)), notrans=factor(notrans, levels=c(0,1)),
#                         cancer=factor(cancer, levels=c(0,1)), diabetes=factor(diabetes, levels=c(0,1)),
#                         pvasc=factor(pvasc, levels=c(0,1)), cva=factor(cva, levels=c(0,1)), 
#                         smoke=factor(smoke, levels=c(0,1)), alcoh=factor(alcoh, levels=c(0,1)),drug=factor(drug, levels=c(0,1)), inci_one=factor(inci_one, levels=c(0,1)),
#                         inci_miss=factor(inci_miss, levels=c(0,1)))


# #COX PROPORTIONAL HAZARDS MODEL 1
# if(test==1){
#   coxph_control <- coxph.control(eps = 1e-8, iter.max = imax)
# } else
# {
#   coxph_control <- coxph.control(eps = 1e-8)
# }

data_sub_sort['year'] = pd.Categorical(data_sub_sort['year'], ordered=False)
data_sub_sort['pdismiss'] = pd.Categorical(data_sub_sort['pdismiss'], categories=[0, 1], ordered=False)
data_sub_sort['pnhprev'] = pd.Categorical(data_sub_sort['pnhprev'], categories=[0, 1], ordered=False)
data_sub_sort['bmi_msg'] = pd.Categorical(data_sub_sort['bmi_msg'], categories=[0, 1], ordered=False)
data_sub_sort['ashd1'] = pd.Categorical(data_sub_sort['ashd1'], categories=[0, 1], ordered=False)
data_sub_sort['othcardiac1'] = pd.Categorical(data_sub_sort['othcardiac1'], categories=[0, 1], ordered=False)
data_sub_sort['carfail'] = pd.Categorical(data_sub_sort['carfail'], categories=[0, 1], ordered=False)
data_sub_sort['noambul'] = pd.Categorical(data_sub_sort['noambul'], categories=[0, 1], ordered=False)
data_sub_sort['pulmon'] = pd.Categorical(data_sub_sort['pulmon'], categories=[0, 1], ordered=False)
data_sub_sort['notrans'] = pd.Categorical(data_sub_sort['notrans'], categories=[0, 1], ordered=False)
data_sub_sort['cancer'] = pd.Categorical(data_sub_sort['cancer'], categories=[0, 1], ordered=False)
data_sub_sort['diabetes'] = pd.Categorical(data_sub_sort['diabetes'], categories=[0, 1], ordered=False)
data_sub_sort['pvasc'] = pd.Categorical(data_sub_sort['pvasc'], categories=[0, 1], ordered=False)
data_sub_sort['cva'] = pd.Categorical(data_sub_sort['cva'], categories=[0, 1], ordered=False)
data_sub_sort['smoke'] = pd.Categorical(data_sub_sort['smoke'], categories=[0, 1], ordered=False)
data_sub_sort['alcoh'] = pd.Categorical(data_sub_sort['alcoh'], categories=[0, 1], ordered=False)
data_sub_sort['drug'] = pd.Categorical(data_sub_sort['drug'], categories=[0, 1], ordered=False)
data_sub_sort['inci_one'] = pd.Categorical(data_sub_sort['inci_one'], categories=[0, 1], ordered=False)
data_sub_sort['inci_miss'] = pd.Categorical(data_sub_sort['inci_miss'], categories=[0, 1], ordered=False)
data_sub_sort['smoke'] = pd.Categorical(data_sub_sort['smoke'], categories=[0, 1], ordered=False)

data_sub_sort['provfs'] = pd.Categorical(data_sub_sort['provfs'], ordered=False)

# coxph_control <- coxph.control(eps = 1e-8)
# data_sub_sort.transform(lambda x: x + 1)

data_model = data_sub_sort


In [ ]:
data_sub_sort['provfs'].loc[0]

In [ ]:
# Read data 
# Artificail weights logic
#packages survival, RCurl, reshape, dplyr, plyr, devtools, sparklyr, sparkR
#
#h2o.init(max_mem_size = "16g",nthreads = -1)
###############################################################################################################
# data_modelt_Full_Sample<-data_model

# # Artificial weights logic
# data_modelt_Full_Sample <- data_modelt_Full_Sample[rep(row.names(data_modelt_Full_Sample), data_modelt_Full_Sample$wt_trans), ] 
# data_modelt_Full_Sample$provfs<-as.factor(data_modelt_Full_Sample$provfs)

# data_modelt_Full_Sample_hex<-as.h2o(data_modelt_Full_Sample)
# #data_modelt_Full_Sample_hex$provfs<-as.factor(data_modelt_Full_Sample_hex$provfs)
# data_modelt_Full_Sample_hex$agecat6<-as.factor(data_modelt_Full_Sample_hex$agecat6)
# data_modelt_Full_Sample_hex$pyf_period_esrd<-as.factor(data_modelt_Full_Sample_hex$pyf_period_esrd)
# data_modelt_Full_Sample_hex$year<-as.factor(data_modelt_Full_Sample_hex$year)
# #data_modelt_Full_Sample_hex$wt_trans<- as.numeric(data_modelt_Full_Sample_hex$wt_trans)
#######################################################################################################################################################
# Run Stage-1 Cox Model
#######################################################################################################################################################
# predictorsSt <- c("agecat6", "pdiab", "pdismiss","notrans","cancer","diabetes","pvasc","year",
#                   "pnhprev", "logbmi", "bmi_msg","cva","smoke","alcoh","drug","inci_one",
#                   "ashd1", "pulmon","inci_miss","year","othcardiac1",
#                   "carfail", "noambul", "pulmon")

# h2o_modelt_D1_p <- h2o.coxph(
#   x = predictorsSt,
#   event_column = "t_trans0",
#   start_column = "t_start",
#   stop_column = "t_stop",
#   offset_column = "ot_trans",
#   ties = c("breslow"),
#   stratify_by = ("provfs"),
#   interaction_pairs=list(
#     c("agecat6","pdiab"),
#     c("pdiab", "pyf_period_esrd")
#   ),
#   training_frame = data_modelt_Full_Sample_hex)

# coefficients_Stage_1<-h2o_modelt_D1_p@model$coefficients_table$coefficients


#############################################
import h2o
from h2o.estimators.coxph import H2OCoxProportionalHazardsEstimator
h2o.init()

data_modelt_Full_Sample=data_model
data_modelt_Full_Sample.reset_index(drop=True, inplace=True)

data_modelt_Full_Sample_hex = h2o.H2OFrame(data_modelt_Full_Sample)
data_modelt_Full_Sample_hex[data_modelt_Full_Sample_hex["provfs"].isna(), "provfs"] = 0
# data_modelt_Full_Sample_hex["provfs"] = data_modelt_Full_Sample_hex["provfs"].ascharacter()
# data_modelt_Full_Sample_hex["provfs"] = data_modelt_Full_Sample_hex["provfs"].asfactor()
predictorsSt = list(["agecat6", "pdiab", "pdismiss","notrans","cancer",
                "diabetes","pvasc","year", "pnhprev", "logbmi", 
                "bmi_msg","cva","smoke","alcoh","drug",
                "inci_one", "ashd1", "pulmon","inci_miss","year",
                "othcardiac1", "carfail", "noambul", "pulmon"])

interaction_pairs = [   ("agecat6","pdiab"), 
                        ("pdiab", "pyf_period_esrd")]

strr_h2o_moodel = H2OCoxProportionalHazardsEstimator(
    start_column="t_start",
    stop_column="t_stop",
    offset_column="ot_trans",
    ties="breslow",
#     stratify_by=["provfs"],
#     interaction_pairs=interaction_pairs,
    )

strr_h2o_moodel.train(x=predictorsSt,
                y="t_trans0",
                training_frame=data_modelt_Full_Sample_hex)
                        
# coefficients_Stage_1<-strr_h2o_moodel.model.coefficients_table.coefficients

In [ ]:
# writing to csv
pd.DataFrame(data_modelt_Full_Sample_hex["provfs"]).to_csv('provfs.csv')

In [ ]:
data_modelt_Full_Sample_hex['provfs'].types

In [ ]:
strr_h2o_moodel.coefficients_table

In [ ]:
h2o.shutdown()

In [ ]:
data_modelt_Full_Sample_hex['t_trans0'].types

In [ ]:
data_modelt_Full_Sample_hex["t_trans0"]

In [ ]:
len(data_modelt_Full_Sample_hex.col_names)

In [ ]:
data_modelt_Full_Sample_hex.types

# Examples

### Lifeline

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from matplotlib import pyplot as plt
from lifelines import CoxPHFitter
import numpy as np
import pandas as pd
from lifelines.datasets import load_rossi
rossi = load_rossi()
cph = CoxPHFitter()

cph.fit(rossi, 'week', 'arrest')

In [ ]:
rossi

In [ ]:
cph.print_summary(model="untransformed variables", decimals=3)

In [ ]:
cph.check_assumptions(rossi, p_value_threshold=0.05, show_plots=True)

In [ ]:
from lifelines.statistics import proportional_hazard_test

results = proportional_hazard_test(cph, rossi, time_transform='rank')
results.print_summary(decimals=3, model="untransformed variables")

In [ ]:
# Stratification
cph.fit(rossi, 'week', 'arrest', strata=['wexp'])
cph.print_summary(model="wexp in strata")

In [ ]:
cph.check_assumptions(rossi, show_plots=True)

### H2O Samples

In [ ]:
import subprocess
import sys
import os

f = os.popen('sudo su; export PATH=$PATH:/opt/h2o; source ~/.bashrc')

In [ ]:
# Airline example
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/interaction_pairs.html
import h2o
h2o.init()
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

# import the airlines dataset
df_air = h2o.import_file("https://s3.amazonaws.com/h2o-public-test-data/smalldata/airlines/allyears2k_headers.zip")

# specify the columns to include
XY = [df_air.names[i-1] for i in [1,2,3,4,6,8,9,13,17,18,19,31]]

# specify the predictor column indices to interact
interactions = [XY[i-1] for i in [5,7,9]]

# train the model and build the coefficients table
m = H2OGeneralizedLinearEstimator(lambda_search=True,
                                  family="binomial",
                                  interactions=interactions)
m.train(x=XY[:len(XY)], y=XY[-1],training_frame=df)
coef_m = m._model_json['output']['coefficients_table']

# define specific interaction pairs
interaction_pairs = [("CRSDepTime", "UniqueCarrier"),
                     ("CRSDepTime", "Origin"),
                     ("UniqueCarrier", "Origin")]

# train the model with the interaction pairs
mexp = H2OGeneralizedLinearEstimator(lambda_search=True,
                                     family="binomial",
                                     interaction_pairs=interaction_pairs)
mexp.train(x=XY[:len(XY)], y=XY[-1],training_frame=df)
coef_mexp = mexp._model_json['output']['coefficients_table']

In [ ]:
df_air.head(5)

In [ ]:
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/coxph.html
import h2o
from h2o.estimators.coxph import H2OCoxProportionalHazardsEstimator
h2o.init()

# Import the heart dataset into H2O:
heart = h2o.import_file("http://s3.amazonaws.com/h2o-public-test-data/smalldata/coxph_test/heart.csv")

# Split the dataset into a train and test set:
train, test = heart.split_frame(ratios = [.8], seed = 1234)

# Build and train the model:
heart_coxph = H2OCoxProportionalHazardsEstimator(start_column="start",
                                                 stop_column="stop",
                                                 ties="breslow")
heart_coxph.train(x="age",
            y="event",
            training_frame=train)

# Generate predictions on a test set (if necessary):
pred = heart_coxph.predict(test)

In [ ]:
a=heart["age"].ascharacter()
b=a.asfactor()
b.types

In [ ]:
# https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/stratify_by.html
import h2o
from h2o.estimators import H2OCoxProportionalHazardsEstimator
h2o.init()

# import the heart dataset:
heart = h2o.import_file("http://s3.amazonaws.com/h2o-public-test-data/smalldata/coxph_test/heart.csv")

# set the predictor and response column:
x = ["age", "year"]
y = "event"

# convert the age column to a factor:
heart["age"] = heart["age"].ascharacter()
heart["age"] = heart["age"].asfactor()

# build and train your model:
heart_coxph = H2OCoxProportionalHazardsEstimator(start_column="start",
                                                 stop_column="stop",
                                                 ties="breslow",
                                                 stratify_by=["age"])
heart_coxph.train(x=x, y=y, training_frame=heart)

# view the model details:
heart_coxph

In [ ]:
#############################################################
# Creation of Dummy variables to match Therneau model
#########################################################
data_modelt<-data_model
#large data set data_modelt

data_modelt$agecat62 <- ifelse(data_modelt$agecat6==2, 1,0) 
data_modelt$agecat63 <- ifelse(data_modelt$agecat6==3, 1,0)
data_modelt$agecat64 <- ifelse(data_modelt$agecat6==4, 1,0) 
data_modelt$agecat66 <- ifelse(data_modelt$agecat6==6, 1,0) 


data_modelt$agecat62p <- (ifelse(data_modelt$agecat6==2, 1,0))*data_modelt$pdiab
data_modelt$agecat63p <- (ifelse(data_modelt$agecat6==3, 1,0))*data_modelt$pdiab
data_modelt$agecat64p <- (ifelse(data_modelt$agecat6==4, 1,0))*data_modelt$pdiab
data_modelt$agecat66p <- (ifelse(data_modelt$agecat6==6, 1,0))*data_modelt$pdiab


data_modelt$pyf_period_esrd2p <- (ifelse(data_modelt$pyf_period_esrd==2, 1,0))*data_modelt$pdiab 
data_modelt$pyf_period_esrd3p <- (ifelse(data_modelt$pyf_period_esrd==3, 1,0))*data_modelt$pdiab 
data_modelt$pyf_period_esrd4p <- (ifelse(data_modelt$pyf_period_esrd==4, 1,0))*data_modelt$pdiab 
data_modelt$pyf_period_esrd5p <- (ifelse(data_modelt$pyf_period_esrd==5, 1,0))*data_modelt$pdiab 
data_modelt$pyf_period_esrd6p <- (ifelse(data_modelt$pyf_period_esrd==6, 1,0))*data_modelt$pdiab 

data_modelt$year2016 <- ifelse(data_modelt$year=="2016", 1,0) 
data_modelt$year2017 <- ifelse(data_modelt$year=="2017", 1,0)
data_modelt$year2018 <- ifelse(data_modelt$year=="2018", 1,0) 


data_modelt$inci_miss1<-as.numeric(as.character(data_modelt$inci_miss))
data_modelt$pdismiss1<-as.numeric(as.character(data_modelt$pdismiss))
data_modelt$bmi_msg1<-as.numeric(as.character(data_modelt$bmi_msg))
data_modelt$ashd11<-as.numeric(as.character(data_modelt$ashd1))

data_modelt$othcardiac11<-as.numeric(as.character(data_modelt$othcardiac1))
data_modelt$carfail1<-as.numeric(as.character(data_modelt$carfail))
data_modelt$noambul1<-as.numeric(as.character(data_modelt$noambul))
data_modelt$pulmon1<-as.numeric(as.character(data_modelt$pulmon))

data_modelt$notrans1<-as.numeric(as.character(data_modelt$notrans))
data_modelt$cancer1<-as.numeric(as.character(data_modelt$cancer))
data_modelt$diabetes1<-as.numeric(as.character(data_modelt$diabetes))
data_modelt$pvasc1<-as.numeric(as.character(data_modelt$pvasc))

data_modelt$cva1<-as.numeric(as.character(data_modelt$cva))
data_modelt$smoke1<-as.numeric(as.character(data_modelt$smoke))
data_modelt$alcoh1<-as.numeric(as.character(data_modelt$alcoh))
data_modelt$drug1<-as.numeric(as.character(data_modelt$drug))

data_modelt$inci_one1<-as.numeric(as.character(data_modelt$inci_one))
data_modelt$pnhprev1<-as.numeric(as.character(data_modelt$pnhprev))


In [ ]:
###################################################################################
# Matrix Reformulation of data_modelt using dummy variables  -  model1
####################################################################################

xf50<-as.matrix(cbind(data_modelt[, "agecat62"], data_modelt[, "agecat63"],data_modelt[, "agecat64"],data_modelt[, "agecat66"],
                      data_modelt[, "year2016"], data_modelt[, "year2017"],data_modelt[, "year2018"], data_modelt[, "bmi_msg1"],
                      data_modelt[, "pdismiss1"],data_modelt[, "pnhprev1"],
                      data_modelt[, "ashd11"], data_modelt[, "othcardiac11"],data_modelt[, "carfail1"],data_modelt[, "noambul1"], 
                      data_modelt[, "pulmon1"],data_modelt[, "notrans1"],data_modelt[, "cancer1"],data_modelt[, "diabetes1"],
                      data_modelt[, "pvasc1"],data_modelt[, "cva1"], data_modelt[, "smoke1"],data_modelt[, "alcoh1"],
                      data_modelt[, "drug1"], data_modelt[, "inci_one1"],data_modelt[, "inci_miss1"],
                      data_modelt[, "agecat62p"],data_modelt[, "agecat63p"], data_modelt[, "agecat64p"],data_modelt[, "agecat66p"],
                      data_modelt[, "pyf_period_esrd2p"],data_modelt[, "pyf_period_esrd3p"],data_modelt[, "pyf_period_esrd4p"],data_modelt[, "pyf_period_esrd5p"], data_modelt[, "pyf_period_esrd6p"],
                      data_modelt[, "pdiab"],data_modelt[, "logbmi"]
))
############################################################
# H2O Model coefficients rearranged to match survival model
############################################################

#Coef <- read_csv("Z:/Divya/h2o/Coef.csv")

#H2OCox<-Coef

#names(H2OCox)[2]<-"coef"

#names(H2OCox)[1]<-"var"

#H2OCoxT<-H2OCox[c(1,2,3,4,17:21,5,6,7,22:36,12:16,8:11),]

In [ ]:
#######################################################################################################
# Linear Predictor Calculations - from Terry Therneau equation - Model Means taken from H2O object
###############################################################################################



lpH2O<- c(xf50%*%coefficients_Stage_1) + data_modelt$ot_trans - sum(coefficients_Stage_1*colMeans(xf50))


###########################################################
# H2O Model Comparison using A. Weights Breslow H2O Cox PH Run - Best Model results
################################################################

modelH2OAWB<- coxph(Surv(t_start, t_stop, t_trans0)~1+ offset(lpH2O),
                    data=data_modelt, weights=wt_trans, ties="breslow", control=coxph_control)


data_modeltH2OAWB<-data_modelt

data_modeltH2OAWB$resid <- residuals(modelH2OAWB,type="martingale")

data_modeltH2OAWB$expecttr <- data_modeltH2OAWB$wt_trans*(data_modeltH2OAWB$t_trans0-data_modeltH2OAWB$resid)

#output expected number of transfusions along with id variables
expect_outH2OAWB<- data_modeltH2OAWB[ ,c("provfs","ptnt_id","year","strr_pyf_start","expecttr")]


In [ ]:
########################################################################################
# SECTION 14: SUMMARIZE TO FACILITY LEVEL AND OUTPUT STrR FOR EACH FACILITY COX H2O Model        #
########################################################################################

sub.dataH2OAWB <- data_modeltH2OAWB[ ,c("provfs","t_trans","expecttr")]

agg.dataH2OAWB <- aggregate(sub.dataH2OAWB[ ,-1], by=list(sub.dataH2OAWB$provfs), "sum")

sum.dataH2OAWB <- agg.dataH2OAWB[ ,c("Group.1","t_trans","expecttr")]

for (i in 1:nrow(sum.dataH2OAWB)){
  if (sum.dataH2OAWB$expecttr[i]>0) {
    sum.dataH2OAWB$strr[i] <- sum.dataH2OAWB$t_trans[i]/sum.dataH2OAWB$expecttr[i]
  }
}


colnames(sum.dataH2OAWB)[1] <- "provfs"
#print(sum.dataH2OAWB)

#boxplot(sum.dataH2OAWBCompare$strr_H2O_AWB, sum.dataH2OAWBCompare$strr_Legacy, names=c("H2O AWB Cox","Legacy Cox"),main="Comparison of STrR Ratios for H2O Cox vs Legacy Cox Model",ylim=c(0,3))

### Sparkling Test

In [ ]:
from pyspark.sql import SparkSession
# from pyspark import SparkConf, SparkContext
# import h2o
from pysparkling import *

spark = SparkSession\
    .builder\
    .appName("H2O_Test")\
    .getOrCreate()
conf = H2OConf().setExternalClusterMode().useManualClusterStart().setCloudName("test")
hc = H2OContext.getOrCreate(conf)

In [ ]:
spark.close()

## Power Plant Model

In [ ]:
# https://github.com/jakubhava/automl_blog_post

from pyspark.sql import SparkSession
from pysparkling import *

# spark = SparkSession\
#     .builder\
#     .appName("GeneralizedLinearRegressionExample")\
#     .getOrCreate()
spark = SparkSession.builder.appName("PowerPlantExample").getOrCreate()
conf = H2OConf().setExternalClusterMode().useManualClusterStart().setCloudName("test")
hc = H2OContext.getOrCreate()

In [ ]:
powerplant_df = spark.read.option("inferSchema", "true").csv("powerplant_output.csv", header=True)
splits = powerplant_df.randomSplit([0.8, 0.2], 1)
train = splits[0]
for_predictions = splits[1]

In [ ]:
from pysparkling.ml import H2OAutoML
from pyspark.ml import Pipeline
from pyspark.ml.feature import SQLTransformer
temperatureTransformer = SQLTransformer(statement="SELECT * FROM __THIS__ WHERE TemperatureCelcius > 10")

In [ ]:
temperatureTransformer.transform(powerplant_df).show()

In [ ]:
automlEstimator = H2OAutoML(maxRuntimeSecs=60, predictionCol="HourlyEnergyOutputMW")

In [ ]:
pipeline = Pipeline(stages=[temperatureTransformer.transform(powerplant_df), automlEstimator])
# pipeline = Pipeline(stages=[automlEstimator])

In [ ]:
powerplant_df.head(2)

In [ ]:
train.head(2)

In [ ]:
for_predictions.head(2)

In [ ]:
model = automlEstimator.fit(train)

In [ ]:
from pyspark.ml.regression import AFTSurvivalRegression 
from pyspark.ml.linalg import Vectors

In [ ]:

from pyspark.sql import SparkSession
# $example on$
from pyspark.ml.regression import GeneralizedLinearRegression
# $example off$

if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("GeneralizedLinearRegressionExample")\
        .getOrCreate()

    # $example on$
    # Load training data
    dataset = spark.read.format("libsvm")\
        .load("s3://eqrs-ngmc-datascience/Datascience/x.csv")

In [ ]:
from pyspark.sql import SparkSession
# $example on$
from pyspark.ml.regression import GeneralizedLinearRegression
# $example off$

if __name__ == "__main__":
    spark = SparkSession\
        .builder\
        .appName("GeneralizedLinearRegressionExample")\
        .getOrCreate()

    # $example on$
    # Load training data
    dataset = spark.read.format("libsvm")\
        .load("sample_linear_regression_data.txt")

    family = ("gaussian","Gamma")
    link=("identity", "Log", "Inverse")
    maxIter=(10,100,1000)
    regParam=(0.1, 0.3, 0.5)
    params = [[i, j, k, l] for i in family  
                     for j in link 
                     for k in maxIter
                     for l in regParam] 
    for p in params:
        glr = GeneralizedLinearRegression(family=p[0], link=p[1], maxIter=p[2], regParam=p[3])

        # Fit the model
        model = glr.fit(dataset)

        # Print the coefficients and intercept for generalized linear regression model
        print("Coefficients: " + str(model.coefficients))
        print("Intercept: " + str(model.intercept))

        # Summarize the model over the training set and print out some metrics
        print(p);print();print()
        summary = model.summary
        print("Coefficient Standard Errors: " + str(summary.coefficientStandardErrors))
        print("T Values: " + str(summary.tValues))
        print("P Values: " + str(summary.pValues))
        print("Dispersion: " + str(summary.dispersion))
        print("Null Deviance: " + str(summary.nullDeviance))
        print("Residual Degree Of Freedom Null: " + str(summary.residualDegreeOfFreedomNull))
        print("Deviance: " + str(summary.deviance))
        print("Residual Degree Of Freedom: " + str(summary.residualDegreeOfFreedom))
        print("AIC: " + str(summary.aic))
        print("Deviance Residuals: ")
        summary.residuals().show()
        # $example off$

    spark.stop()